In [7]:
from typing import Tuple
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil

In [8]:
# 시스템의 논리적 CPU 코어 수를 기반으로 워커 수 결정
optimal_workers = psutil.cpu_count(logical=True)

print(f"사용할 워커 수: {optimal_workers}")

# 시스템의 전체 메모리 정보를 가져옴
mem = psutil.virtual_memory()

print(f"총 메모리: {mem.total / (1024**3):.2f} GB")
print(f"사용 가능 메모리: {mem.available / (1024**3):.2f} GB")
print(f"사용 중인 메모리: {mem.used / (1024**3):.2f} GB")
print(f"메모리 사용률: {mem.percent}%")

사용할 워커 수: 64
총 메모리: 125.14 GB
사용 가능 메모리: 112.02 GB
사용 중인 메모리: 13.12 GB
메모리 사용률: 10.5%


In [9]:
# 프로젝트 루트를 sys.path에 추가
project_root = Path.cwd().parent  # code 폴더에서 한 단계 위
print(project_root)
sys.path.append(str(project_root))
sys.path.append(str(project_root / 'code'))

# 이제 import 가능
from loading import DataLoader

/home/dataiku/eric/Sparta/Project3


In [10]:
loader2 = DataLoader(
    start=2020,
    end=2025,
    output_file='../output.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
df = loader2.load(adapter=adapter, **polars_kwargs)
df


📖 ../output.parquet 로딩 중... (adapter=polars)


In [11]:
mdr_max = 79
device_max = 54
patient_max = 3

In [13]:
# 상표명, generic_name
cols = ['device_0_brand_name', 'device_0_generic_name', 'device_0_device_report_product_code']

df.select(pl.col(cols)).group_by('device_0_device_report_product_code').agg(
    pl.col('device_0_generic_name').n_unique().alias('unique_generic_name')
).collect().to_pandas().sort_values('unique_generic_name', ascending=False)

,device_0_device_report_product_code,unique_generic_name
333,JWH,2277
246,DZE,1548
2160,LZO,1113
104,LXH,1091
787,FPA,992
...,...,...
2208,RFN,1
2213,LZW,1
2214,JCE,1
2215,QDF,1


In [14]:
df.select(cols).filter(
    pl.col('device_0_device_report_product_code').str.contains('JWH')
).group_by('device_0_generic_name').agg(
    pl.col('device_0_generic_name').len().alias('cnt_generic_name'),
    pl.col('device_0_brand_name').mode().alias('mode_brand_name')
).collect().to_pandas().sort_values('cnt_generic_name', ascending=False)

,device_0_generic_name,cnt_generic_name,mode_brand_name
1304,"PROSTHESIS, KNEE, PATELLOFEMOROTIBIAL, SEMI-CO...",10595,[SPECIFIC DEVICE NOT REPORTED]
2276,"PROSTHESIS, KNEE",9274,[UNKNOWN ARTICULAR SURFACE]
1645,"INSTRUMENT, KNEE",2653,[FEMORAL CR IMPACTOR PAD]
1924,KNEE TIBIAL INSERT,2436,[UNKNOWN KNEE TIBIAL INSERT]
2249,KNEE TIBIAL TRAY,1924,[UNKNOWN KNEE TIBIAL TRAY]
...,...,...,...
28,KNEE SEMICONSTRAINED INSERT,1,[GMK-REVISION FIXED TIBIAL INSERT SC SIZE 2/14MM]
14,"SOMBRERO TRI-PEG PATELLA, 9MM X 38MM",1,[MTS KNEE]
15,UNCEMENTED FIXED BEARING TIBIAL TRAYS - BICOMP...,1,[ANATOMIC]
16,KNEE UNIVERSAL SLEEVE,1,[UNKNOWN KNEE UNIVERSAL STEM]


In [ ]:
manu_cond = True
manu_cond &= pl.col('manufacturer_g1_name') != 'None'
manu_cond &= pl.col('manufacturer_g1_name') != 'UNK'

manu_filtered = df.filter(manu_cond)

columns = manu_filtered.collect_schema().names()

# 간단하게 streaming 모드만 활성화
cols_to_keep = []
batch_size = 100

for i in range(0, len(columns), batch_size):
    batch_cols = columns[i:i + batch_size]
    
    result = manu_filtered.select([
        pl.col(col).is_not_null().any().alias(col)
        for col in batch_cols
    ]).collect(engine='streaming')
    
    cols_to_keep.extend([
        col for col in batch_cols if result[col][0]
    ])

manu_filtered = manu_filtered.select(cols_to_keep)
total_rows = manu_filtered.select(pl.len()).collect()

In [37]:
unk_lst = df.filter(pl.col('device_0_model_number').str.to_uppercase().str.contains(r'^UNK')).select('device_0_model_number').collect().to_series().to_list()
set(unk_lst)

{'UNK',
 'UNK - COOLTIP ANT',
 'UNK - I/A HANDPIECE',
 'UNK - PILLCAM',
 'UNK - PRODUCT - CORDIS CORPORATION',
 'UNK - SPECTRUM',
 'UNK - VESSEL DILATOR',
 'UNK -I/A HANDPIECE',
 'UNK 014 ALLSTAR',
 'UNK 014 BHW',
 'UNK 014 BMW HYDRO',
 'UNK 014 FLOPPY HYDRO',
 'UNK 014 HI-TORQUE PILOT',
 'UNK 014 HT TURNTRAC ES',
 'UNK 014 IRONMAN HYDRO',
 'UNK 014 WHISPER ES',
 'UNK ABSOLUTE PRO',
 'UNK ABSOLUTE PRO LL',
 'UNK ABSOLUTE PRO VASCULAR',
 'UNK ABSORB',
 'UNK ALARIS PUMP MODULE',
 'UNK AMPLATZER AMULET',
 'UNK AMPLATZER AMULET DS',
 'UNK AMPLATZER CARDIAC PLUG',
 'UNK AMPLATZER DUCT OCCL II, AS',
 'UNK AMPLATZER DUCT OCCLUDER',
 'UNK AMPLATZER DUCT OCCLUDER II',
 'UNK AMPLATZER FSO CRIBRIFORM',
 'UNK AMPLATZER OCCLUDER',
 'UNK AMPLATZER PFO OCCLUDER',
 'UNK AMPLATZER PICCOLO',
 'UNK AMPLATZER SEPTAL OCCLUDER',
 'UNK AMPLATZER TORQVUE',
 'UNK AMPLATZER VASCULAR PLUG',
 'UNK AMPLATZER VASCULAR PLUG II',
 'UNK AMPLATZER VASCULAR PLUG IV',
 'UNK AMPLATZERVASCULARPLUG III',
 'UNK ARMADA',
 'UN

In [ ]:
def filter_notna_rows(df: pl.LazyFrame, col) -> Tuple[pl.LazyFrame, pl.Expr, pl.DataFrame]:
    na_patterns = r'^None$|^UNK|^NOT APPLICABLE$|^N/A$|^NA$|^$|\s+$'
    notna_cond = True
    notna_cond &= ~pl.col(col).str.to_uppercase().str.contains(na_patterns)
    filtered = df.filter(notna_cond)
    notna_rows = filtered.select(pl.len()).collect()
    return filtered, notna_cond, notna_rows

In [39]:
total_rows = df.select(pl.len()).collect().item()

udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, 'device_0_udi_di')
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, 'device_0_model_number')

coverage_analysis = df.select([
    udi_notna_cond.is_not_null().alias("has_udi"),
    model_notna_cond.is_not_null().alias("has_model")
]).group_by(["has_udi", "has_model"]).len().with_columns([
    pl.format("{}%", (pl.col("len").mul(100).truediv(total_rows).round(1)).alias("pct"))
])

# 실제 커버리지 계산
actual_coverage = df.select([
    (udi_notna_cond | model_notna_cond).mean().alias("either"),
    (udi_notna_cond & model_notna_cond).mean().alias("both")
])

coverage_analysis.collect(), actual_coverage.collect()

(shape: (4, 4)
 ┌─────────┬───────────┬─────────┬───────┐
 │ has_udi ┆ has_model ┆ len     ┆ pct   │
 │ ---     ┆ ---       ┆ ---     ┆ ---   │
 │ bool    ┆ bool      ┆ u32     ┆ str   │
 ╞═════════╪═══════════╪═════════╪═══════╡
 │ true    ┆ false     ┆ 1754223 ┆ 12.9% │
 │ true    ┆ true      ┆ 7471374 ┆ 54.8% │
 │ false   ┆ false     ┆ 1661070 ┆ 12.2% │
 │ false   ┆ true      ┆ 2755969 ┆ 20.2% │
 └─────────┴───────────┴─────────┴───────┘,
 shape: (1, 2)
 ┌────────┬──────────┐
 │ either ┆ both     │
 │ ---    ┆ ---      │
 │ f64    ┆ f64      │
 ╞════════╪══════════╡
 │ 1.0    ┆ 0.935589 │
 └────────┴──────────┘)

In [40]:
df.filter(udi_notna_cond & ~model_notna_cond).first().select('device_0_udi_di').collect()

device_0_udi_di
str
"""00389701008851"""


In [41]:
device_cols = [f'device_{i}_udi_di' for i in range(50)]
model_cols = [f'device_{i}_model_number' for i in range(50)]

target_cols = device_cols
unique_values = set()

for c in target_cols:
    lst = df.select(pl.col(c).unique())
    unique_values.update(lst.collect().to_series().to_list())

print(len(unique_values))

142336


In [42]:
df.filter(pl.col('device_0_brand_name').str.to_uppercase().str.starts_with('ACCELERATOR')).collect().to_pandas()

,adverse_event_flag,date_added,date_changed,date_facility_aware,date_manufacturer_received,date_of_event,date_received,date_report,date_report_to_fda,date_report_to_manufacturer,...,report_to_manufacturer,reporter_country_code,reporter_occupation_code,reprocessed_and_reused_flag,single_use_flag,source_type,summary_report_flag,suppl_dates_fda_received,suppl_dates_mfr_received,type_of_report
0,Y,20230901,20240409,None,20230804,None,20230901,20231012,None,None,...,None,US,003,N,N,['Distributor'],N,10/12/2023,09/14/2023,"['Initial submission', 'Followup']"
1,Y,20230831,20230907,None,None,None,20230829,20230825,None,None,...,None,US,OTHER,None,None,None,N,None,None,['Initial submission']
2,N,20201210,20201224,None,20201112,20201106,20201210,20201210,None,None,...,None,US,003,N,N,['DISTRIBUTOR'],N,None,None,['Initial submission']
3,N,20201021,20230803,None,20200925,None,20201021,20201116,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,11/16/2020,11/06/2020,"['Initial submission', 'Followup']"
4,N,20201016,20230808,None,20200924,None,20201016,20201105,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,11/05/2020,11/05/2020,"['Initial submission', 'Followup']"
5,Y,20250903,20251021,None,20250804,20250804,20250903,20251021,None,None,...,None,US,003,N,None,['Distributor'],N,10/21/2025,09/23/2025,"['Initial submission', 'Followup']"
6,N,20211019,20231024,None,20210929,20210929,20211019,20211213,None,None,...,None,CA,OTHER HEALTH CARE PROFESSIONAL,N,N,"['Foreign', 'Health Professional']",N,12/13/2021,11/30/2021,"['Initial submission', 'Followup']"
7,N,20211122,20231130,None,20211027,20211027,20211122,20220606,None,None,...,None,US,OTHER HEALTH CARE PROFESSIONAL,N,N,['Health Professional'],N,05/04/2022;06/06/2022,04/07/2021;06/06/2022,"['Initial submission', 'Followup', 'Followup']"
8,N,20200416,20200417,None,20200326,20200326,20200416,20200416,None,None,...,None,US,OTHER,N,N,['COMPANY REPRESENTATIVE'],N,None,None,['Initial submission']
9,N,20200413,20230531,None,20200325,None,20200413,20200512,None,None,...,None,None,None,N,None,"['FOREIGN', 'HEALTH PROFESSIONAL']",N,05/12/2020,04/30/2020,"['Initial submission', 'Followup']"


In [43]:
df.select(pl.col('patient_0_patient_sequence_number', 'patient_1_patient_sequence_number', 'patient_2_patient_sequence_number', 'patient_3_patient_sequence_number').is_not_null().sum()).collect()

patient_0_patient_sequence_number,patient_1_patient_sequence_number,patient_2_patient_sequence_number,patient_3_patient_sequence_number
u32,u32,u32,u32
13637345,117,2,1


In [44]:
total_rows = df.select(pl.len()).collect()

manu_col = 'device_0_manufacturer_d_name'
udi_col = 'device_0_udi_di'
model_col = 'device_0_model_number'
manu_filtered, manu_notna_cond, manu_notna_rows = filter_notna_rows(df, manu_col)
udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, udi_col)
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, model_col)

print(manu_notna_rows / total_rows * 100)
print(udi_notna_rows / total_rows * 100)
print(model_notna_rows / total_rows * 100)

shape: (1, 1)
┌──────────┐
│ len      │
│ ---      │
│ f64      │
╞══════════╡
│ 99.38152 │
└──────────┘
shape: (1, 1)
┌───────────┐
│ len       │
│ ---       │
│ f64       │
╞═══════════╡
│ 67.623273 │
└───────────┘
shape: (1, 1)
┌───────────┐
│ len       │
│ ---       │
│ f64       │
╞═══════════╡
│ 71.288855 │
└───────────┘


In [45]:
total_rows = manu_filtered.select(pl.len()).collect()
manu_filtered.select('device_0_udi_di').null_count().collect() / total_rows

device_0_udi_di
f64
0.322627


In [46]:
columns = manu_filtered.collect_schema().names()
pprint(columns)

['adverse_event_flag',
 'date_added',
 'date_changed',
 'date_facility_aware',
 'date_manufacturer_received',
 'date_of_event',
 'date_received',
 'date_report',
 'date_report_to_fda',
 'date_report_to_manufacturer',
 'device',
 'device_0_brand_name',
 'device_0_catalog_number',
 'device_0_combination_product_flag',
 'device_0_date_received',
 'device_0_date_removed_flag',
 'device_0_date_returned_to_manufacturer',
 'device_0_device_age_text',
 'device_0_device_availability',
 'device_0_device_evaluated_by_manufacturer',
 'device_0_device_event_key',
 'device_0_device_operator',
 'device_0_device_report_product_code',
 'device_0_device_sequence_number',
 'device_0_expiration_date_of_device',
 'device_0_generic_name',
 'device_0_implant_flag',
 'device_0_lot_number',
 'device_0_manufacturer_d_address_1',
 'device_0_manufacturer_d_address_2',
 'device_0_manufacturer_d_city',
 'device_0_manufacturer_d_country',
 'device_0_manufacturer_d_name',
 'device_0_manufacturer_d_postal_code',
 'dev

In [47]:

mdr_cond = pl.col('mdr_text_79_text') != 'None'
device_cond = pl.col('device_47_device_report_product_code') != 'None'
patient_cond = pl.col('patient_3_patient_sequence_number') != 'None'

pprint(manu_filtered.filter(device_cond).collect().to_dicts())
# df.filter(pl.col('device_54_device_report_product_code') != 'None').collect()
# df.filter(pl.col('patient_3_patient_sequence_number') != 'None').collect()

[{'adverse_event_flag': 'Y',
  'date_added': '20221222',
  'date_changed': '20230125',
  'date_facility_aware': None,
  'date_manufacturer_received': None,
  'date_of_event': None,
  'date_received': '20221219',
  'date_report': '20221215',
  'date_report_to_fda': None,
  'date_report_to_manufacturer': None,
  'device': None,
  'device_0_brand_name': 'CASSETTE MEDICATION RESERVOIR',
  'device_0_catalog_number': None,
  'device_0_combination_product_flag': 'Y',
  'device_0_date_received': '20221219',
  'device_0_date_removed_flag': None,
  'device_0_date_returned_to_manufacturer': None,
  'device_0_device_age_text': None,
  'device_0_device_availability': 'No answer provided',
  'device_0_device_evaluated_by_manufacturer': None,
  'device_0_device_event_key': None,
  'device_0_device_operator': 'LAY USER/PATIENT',
  'device_0_device_report_product_code': 'LHI',
  'device_0_device_sequence_number': '1',
  'device_0_expiration_date_of_device': None,
  'device_0_generic_name': 'SET, I.V. F